In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[58]:


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings

warnings.filterwarnings("ignore",category =RuntimeWarning)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_columns',500)
pd.set_option('chained_assignment',None)


# In[87]:


#Reading Raw Data file
loan_raw = pd.read_csv("loan.csv",parse_dates = [15,26],low_memory=False)
loan_raw.info()


# ### Data Facts

# In[325]:


print ("Number of Loan Customers in the data: ",loan_raw.member_id.nunique())
print ("Percent of Loan customers who defaulted,fully paid and are currently active loan payers \n: ",loan_raw.groupby('loan_status')['member_id'].nunique()/loan_raw.member_id.nunique()*100)


# ### checking and dropping columns with 90% data null

# In[88]:



print(loan_raw.isnull().sum()/len(loan_raw.index)*100)
loan_f1 = loan_raw.dropna(thresh=0.9*len(loan_raw), axis=1)
print(loan_f1.isnull().sum()/len(loan_raw.index)*100)


# ### getting the Numeric and Obeject type columns

# In[89]:



objects = loan_f1.select_dtypes(include=['object']).columns.values
numbers = loan_f1._get_numeric_data().columns.values


# In[90]:


#Understanding and analyzing the Object/Categorical columns
display(loan_f1[objects].head())
display(loan_f1[objects].describe())


# #### Int_rate and revol_util columns should be numeric, removing the % from data and converting to float and renaming the columns to include % as part of column label

# In[91]:



loan_f1['int_rate'] = loan_f1['int_rate'].apply(lambda x: x.replace('%',''))
loan_f1['int_rate'] = loan_f1['int_rate'].astype(float)
loan_f1['revol_util'] = loan_f1['revol_util'].astype(str).apply(lambda x: x.replace('%',''))
loan_f1['revol_util'] = loan_f1['revol_util'].astype(float)
loan_f1.rename(columns = {'int_rate':'int_rate(%)','revol_util':'revol_util(%)'},inplace=True)



# #### We see that of all the object variables pymnt_plan, initial_list_status and application_type have same value across the data and hence would provide little help in differentiating defaulters and non-defaulters. Also URL has all distinct values and looking at the sample suggests that its not a useful metric here. emp_title and title has a big range of unique values, it will create far too many buckets to segment the data and extract any usefull pattern. Dropping these

# In[ ]:


loan_f1.drop(columns = ['pymnt_plan','initial_list_status','application_type','url','emp_title','title'],axis = 1,inplace=True)


# In[92]:


#getting the Numeric and Obeject type columns again, after the change
objects = loan_f1.select_dtypes(include=['object']).columns.values
numbers = loan_f1._get_numeric_data().columns.values
loan_f1[objects].describe()


# ##### Lets understand the distribution of data in these categorical variables, using univariate analysis, using plots

# In[290]:



kwargs = dict(histtype='bar', alpha=0.5, normed=False, bins = 8,color= 'g',edgecolor='black',linewidth=1.2)
plt.figure(figsize=(20,20))
plt.subplot(3,3,1)
plt.ylabel('Count')
plt.xlabel('term')
loan_f1['term'].value_counts().plot(kind='bar',rot=0)

plt.subplot(3,3,2)
plt.ylabel('Count')
plt.xlabel('grade')
loan_f1['grade'].value_counts().plot(kind='bar')


plt.subplot(3,3,3)
plt.ylabel('Count')
plt.xlabel('emp_length')
loan_f1['emp_length'].value_counts().plot(kind='bar',rot= 25)

plt.subplot(3,3,4)
plt.ylabel('Count')
plt.xlabel('home_ownership')
loan_f1['home_ownership'].value_counts().plot(kind='bar')

plt.subplot(3,3,5)
plt.ylabel('Count')
plt.xlabel('verification_status')
loan_f1['verification_status'].value_counts().plot(kind='bar')

plt.subplot(3,3,6)
plt.ylabel('Count')
plt.xlabel('loan_status')
#plt.xscale('log')
loan_f1['loan_status'].value_counts().plot(kind='bar')

plt.show()


# In[289]:


plt.figure(figsize=(8,6))
plt.subplot(1,1,1)
plt.ylabel('Count')
plt.xlabel('purpose')
#plt.xscale('log')
#plt.yscale('log')
loan_f1['purpose'].value_counts().plot(kind='bar')
plt.show()


# In[95]:


loan_f1.info()


# In[383]:


#Lets try to understand the number of loans distribution by year and month, this needs derived variables Month
#and year from issue_d column as well as earliest_cr_line
loan_f1['issue_month'] = loan_f1.issue_d.str[0:3]
loan_f1['issue_year'] = loan_f1.issue_d.str[4:]


# In[393]:


fig, ax = plt.subplots(figsize=(18,6))
loan_by_month = loan_f1.groupby(['issue_year','issue_month'])

#fig,ax = plt.subplots()
avgLoanSizeByMonth = loan_by_month['funded_amnt_inv'].agg(np.mean).plot(ax=ax,label='issue_year')

avgLoanSizeByMonth.set_title('Avg. Loan Size By Month')
plt.show()



'''fig, ax = plt.subplots(figsize=(18,6))

for name, group in loan_f1.groupby(['issue_year','issue_month']):
    group.plot(x=name, y='funded_amnt_inv', ax=ax)

plt.show()'''


# ### Lets look at the Numerical Variables

# In[98]:



display(loan_f1[numbers].head())
display(loan_f1[numbers].describe())


# #### We see that columns 'collections_12_mths_ex_med',	'acc_now_delinq',	'chargeoff_within_12_mths',	'tax_liens', 'delinq_amnt'  have min,max and mean as 0, making it useless to differentiate between defaulters and non-defaulters. id, member id columns are also quite useless for the purpose

# In[99]:


loan_f1.drop(columns = ['collections_12_mths_ex_med','acc_now_delinq','chargeoff_within_12_mths','tax_liens','id','member_id','delinq_amnt'],axis = 1,inplace=True)


# In[100]:


numbers = loan_f1._get_numeric_data().columns.values
display(loan_f1[numbers].describe())


# #### Loan status with value 'current' will not be of use for us, as we are interested in charge off v/s fully paids.  Lets drop that and then see how these variables behave with loan_status

# In[374]:


loan_f2 = loan_f1[loan_f1['loan_status']!='Current']


# #### we see that loan_amnt,funded_amnt,funded_amnt_inv have  close medians and IQR, these might be correlated. lest check

# In[119]:


loan_f2[['loan_amnt','funded_amnt','funded_amnt_inv']].corr()


# In[394]:


#Correlation plots of all numeric variables
corr = loan_f2[numbers].corr()
plt.figure(figsize=(18, 10))# plot the heatmap
sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns)
plt.show()


# ### lets create 2 new variables as ratio from loan amnt to see if helps differentiate within Loan Status

# In[122]:


loan_f2['ratio_loan_funded'] = loan_f2['funded_amnt']/loan_f2['loan_amnt']
loan_f2['ratio_loan_funded_inv'] = loan_f2['funded_amnt_inv']/loan_f2['loan_amnt']


# ### Bivariate analysis to understand the impact of some numerical variables on Loan Status

# In[307]:


plt.figure(figsize=(25,6))
plt.style.use('ggplot')
plt.subplot(1,3,1)

plt.title("Loan Amnt by Loan Status")
plt.ylabel('Count')
plt.xlabel('Percent change in allocation')
a = sns.boxplot(x='loan_status',y='loan_amnt',data=loan_f2)
a.set(xlabel='loan_status',ylabel = 'loan_amnt')


plt.subplot(1,3,2)
plt.title("Ratio of Funded Amnt to  \n Loan Amnt by Laon Status")
plt.ylabel('Count')
plt.xlabel('Percent change in allocation')
b = sns.boxplot(x='loan_status',y='ratio_loan_funded',data=loan_f2)
b.set(xlabel='loan_status',ylabel = 'funded_amnt')

plt.subplot(1,3,3)
plt.title("Ratio of Funded Amnt Inv to  \n Loan Amnt by Laon Status")
plt.ylabel('Count')
plt.xlabel('Percent change in allocation')
c = sns.boxplot(x='loan_status',y='ratio_loan_funded_inv',data=loan_f2)
c.set(xlabel='loan_status',ylabel = 'funded_amnt_inv')

plt.show()


# ### Trying Segmentation based on some categorical variables and then see imapct using bivariate analysis

# In[346]:


#f, axes = plt.subplots(1, 2)

plt.figure(figsize=(25,6))
plt.style.use('ggplot')
plt.subplot(1,3,1)
plt.title("Interest Rate by Loan Status and term")
sns.boxplot(x='term',y='int_rate(%)',data=loan_f2,hue='loan_status')
plt.legend(loc='upper left')

plt.subplot(1,3,2)
plt.title("Interest Rate by Loan Status and Grade")
sns.boxplot(x='grade',y='int_rate(%)',data=loan_f2,hue='loan_status')
plt.legend(loc='upper left')

'''plt.subplot(1,3,2)
plt.title("Installment by Loan Status")
sns.boxplot(x='loan_status',y='installment',data=loan_f2)


plt.subplot(1,3,3)
plt.title("Annual Income by Loan Status")
plt.yscale('log')
sns.boxplot(x='loan_status',y='annual_inc',data=loan_f2)
'''
plt.show()


# In[ ]:





# In[186]:


display(loan_f2.groupby('loan_status')['pub_rec'].describe()) # may not be useful
display(loan_f2.groupby('loan_status')['pub_rec_bankruptcies'].describe()) #may not be useful

#data looks too skewed where the interquartile range is 0


# In[313]:


plt.figure(figsize=(25,7))
plt.style.use('ggplot')

plt.subplot(2,2,1)
plt.title("Open loan accounts by Loan Status")
a = sns.boxplot(x='loan_status',y='open_acc',data=loan_f2)


plt.subplot(2,2,2)
plt.title("Total Loan Accounts by Loan Status")
sns.boxplot(x='loan_status',y='total_acc',data=loan_f2)

plt.show()


# In[395]:



loan_f2['ratio_open_total'] = loan_f2['open_acc']/loan_f2['total_acc']

plt.figure(figsize=(35,9))

'''plt.subplot(2,2,1)
plt.title("Ratio of Open accouts to total accounts by loan status")
plt.ylabel('Count')
#plt.xlabel('Percent change in allocation')
b = sns.boxplot(x='loan_status',y='ratio_open_total',data=loan_f2)
b.set(xlabel='loan_status',ylabel = 'ratio_open_total')
#plt.show()'''


plt.subplot(1,3,2)
plt.title("Revol Util % by loan status and Term")
plt.ylabel('Count')
#plt.xlabel('Percent change in allocation')
b = sns.boxplot(x='term',y='revol_util(%)',data=loan_f2,hue = 'loan_status')
b.set(xlabel='loan_status',ylabel = 'revol_util(%)')
#plt.show()
plt.legend(loc = 'upper left')

plt.subplot(1,3,3)
plt.title("Revol Util % by loan status and Grade")
plt.ylabel('Count')
#plt.xlabel('Percent change in allocation')
b = sns.boxplot(x='grade',y='revol_util(%)',data=loan_f2,hue = 'loan_status')
b.set(xlabel='loan_status',ylabel = 'revol_util(%)')
#plt.show()
plt.legend(loc = 'upper left')

plt.subplot(1,3,1)
plt.title("Revol Util % by loan status")
plt.ylabel('Count')
#plt.xlabel('Percent change in allocation')
b = sns.boxplot(x='term',y='revol_util(%)',data=loan_f2)
b.set(xlabel='loan_status',ylabel = 'revol_util(%)')
#plt.legend(loc = 'upper left')
plt.show()


# #### Loan v/s Term

# In[196]:


Y=loan_f2['loan_status']
X=loan_f2[["term","grade","sub_grade","emp_length","home_ownership","verification_status","purpose"]]


# In[197]:


loan_vs_terma=pd.crosstab(index=loan_f2['term'], columns=Y, margins=True).round(2)
loan_vs_terma
loan_vs_terma.div(loan_vs_terma['All'],axis=0).round(2)


# In[198]:


loan_vs_terma.div(loan_vs_terma['All'],axis=0).round(2)


# In[212]:


table=(loan_vs_terma/loan_vs_terma.loc["All",'All'].round(2))


plt.figure(figsize=(10, 4))
plt.subplot(1,2,1)
sns.heatmap(table,annot=True,cmap='coolwarm',fmt=".1%",robust=False,annot_kws={"size":12})

plt.subplot(1,2,2)
b=loan_vs_terma.div(loan_vs_terma['All'],axis=0).round(2)
sns.heatmap(b,annot=True,cmap='coolwarm',fmt=".1%",robust=False,annot_kws={"size":12})

chi2, p, dof, expctd=stats.chi2_contingency(loan_vs_terma)
plt.show()
print("chi2 value:",chi2 ,"\n", "P value:",p)


# ### Loan Status v/s Grade

# In[360]:


table=pd.crosstab(index=loan_f2['loan_status'], columns=loan_f2['grade'], margins=True)
a=table/table.loc['All','All'].round(2)
b=table/table.loc['All'].round(2)
plt.figure(figsize=(10, 4))
plt.title("Loan Status by Grades")
sns.heatmap(b,annot=True,cmap='coolwarm',fmt=".1%",center=True,robust=True,annot_kws={"size":12.5},)
plt.show()


# In[214]:


table=pd.crosstab(index=loan_f2['loan_status'], columns=loan_f2['grade'], margins=True)
table.head()


# ### Loan Status v/s Sub-grade

# In[361]:


eda_filter=loan_f2.loc[loan_f2['grade'].isin(['E','F','G'])]
#eda_filter
table_filter=pd.crosstab(index=[eda_filter['grade'],eda_filter['loan_status']],columns=[eda_filter['sub_grade']],margins=True)

#b=table_filter/table_filter.ix['All'].round(2)
b=table_filter.div(table_filter['All'],axis=0)

plt.figure(figsize=(16, 8))
plt.title("Loan Status by Grades and Subgrades")
sns.heatmap(b,annot=True,cmap='coolwarm',fmt=".1%",center=True,robust=True,annot_kws={"size":12.5},)
plt.show()


# ### Loan Status v/s Employement Length (emp_length)

# In[222]:


table_filter=pd.crosstab(index=loan_f2['emp_length'],columns=loan_f2['loan_status'],margins=True)
b=table_filter/table_filter.loc['All'].round(2)
#b=table_filter.div(table_filter['All'])

plt.figure(figsize=(17, 5))
sns.heatmap(b,annot=True,cmap='coolwarm',fmt=".1%",center=True,robust=True,annot_kws={"size":12.5},)
plt.show()


# ### Loan Status v/s Home Ownership

# In[225]:


table_filter=pd.crosstab(index=loan_f2['home_ownership'],columns=loan_f2['loan_status'],margins=True)
#b=table_filter/table_filter.ix['All'].round(2)
b = table_filter / table_filter.loc['All'].round(2)

plt.figure(figsize=(12, 5))

sns.heatmap(b,annot=True,fmt=".1%",center=True,robust=True,annot_kws={"size":12.5})
plt.show()


# ### Loan Status v/s Purpose

# In[359]:


table_filter=pd.crosstab(index=loan_f2['purpose'],columns=loan_f2['loan_status'],margins=True)
#b=table_filter/table_filter.ix['All'].round(2)
b = table_filter / table_filter.loc['All'].round(2)

plt.figure(figsize=(12, 5))
plt.title("Loan Status by purpose of loan")
sns.heatmap(b,annot=True,fmt=".1%",center=True,robust=True,annot_kws={"size":12.5})
plt.show()


# ### Delinq_2yrs v/s Loan Status

# In[356]:


table_filter=pd.crosstab(index=loan_f2[loan_f2['delinq_2yrs']!=0]['delinq_2yrs'],columns=loan_f2[loan_f2['delinq_2yrs']!=0]['loan_status'],margins=True)
table_filter=pd.crosstab(index=loan_f2['delinq_2yrs'],columns=loan_f2['loan_status'],margins=True)

#b=table_filter/table_filter.ix['All'].round(2)
b = table_filter / table_filter.loc['All'].round(2)

plt.figure(figsize=(12, 5))
plt.title("Loan Status by count of delinquency (in last 2 years)")
sns.heatmap(b,annot=True,fmt=".1%",center=True,robust=True,annot_kws={"size":12.5})
plt.show()


# ### Loan Amount segmented by grade and empl length, imapct on loan status

# In[396]:


plt.figure(figsize=(20, 6))
plt.subplot(1,2,1)
plt.title("Laon Amount by Loan Status and employement length")

sns.boxplot(x='emp_length',y='loan_amnt', hue='loan_status',data=loan_f2)
plt.legend(loc = 'upper left')

plt.subplot(1,2,2)
plt.title("Loan Amount by Loan Status and Grade")

#plt.figure(figsize=(10, 4))
sns.boxplot(x='grade',y='loan_amnt', hue='loan_status',data=loan_f2)
plt.legend(loc = 'upper left')
plt.show()


# ### Annual Income, treated for Outliers and then plotted against loan status, segmented by grade

# In[378]:


def replace(var, stds):
    import numpy as np
    var[np.abs(var - var.mean()) > stds * var.std()]= stds * var.std()
    return var
loan_f3 = loan_f2
loan_f3.loc['annual_inc'] = loan_f3['annual_inc'].transform(lambda g : replace(g, 2))
#loan_f3[['annual_inc',"annual_inc_2"]].describe()


plt.figure(figsize=(25, 5))
plt.subplot(1,2,1)
plt.title("Annual Income by Loan Status and grade")

# set figure size for larger figure
#plt.figure(num=None, figsize=(15, 10), dpi=80, facecolor='w', edgecolor='k')

# specify hue="categorical_variable"
sns.boxplot(x='grade', y='annual_inc', hue="loan_status", data=loan_f3[loan_f3['annual_inc']<=120000])
plt.legend(loc = 'upper left')
plt.show()


# ### Revol Util % against Laon Status segmented by Employement Length

# In[244]:


plt.figure(figsize=(20, 4))
#plt.yscale('log')
sns.boxplot(x='emp_length',y='revol_util(%)', hue='loan_status',data=loan_f2)
plt.show()


# ### Revol Util % against Laon Status segmented by Home Ownership

# In[246]:


plt.figure(figsize=(20, 4))
#plt.yscale('log')
sns.boxplot(x='home_ownership',y='revol_util(%)', hue='loan_status',data=loan_f2)
plt.show()


# ### Analysing how charge off cases have varied over the years. It recuced as we moved from 2007 to 2010 and then again increased in 2011

# In[271]:


plt.figure(figsize=(20, 4))
plt.yscale('log')
sns.boxplot(x='issue_year',y='funded_amnt', hue='loan_status',data=loan_f2)
plt.show()


# ### delinq_2yrs against Laon Status.

# In[278]:


loan_f2
plt.figure(figsize=(20, 4))
#plt.yscale('log')
sns.boxplot(x='loan_status',y='delinq_2yrs',data=loan_f2[loan_f2['delinq_2yrs']!=0])
plt.show()

